In [10]:
import json_lines
import re
import json
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
import string
import pandas as pd
import mtranslate

In [11]:
def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None

In [24]:
df = pd.DataFrame(columns=['Post', 'Comment', 'Topic', 'Sentiment', 'Source'])
lemmatizer = WordNetLemmatizer()

#For some reason, please returns positive sentiment because it assumes 'pleasing' but normally people just asking politely
weird_sentiment_words = ['la', 'please', 'i']

file = json.load(open('chifitness6months.json', encoding="utf8"))
for line in file['data']:
    
    try:
        #Check if key exists
        line['comments']
        #print(line['message'])
        
        for comments in line['comments']['data']:
            print("New comment")
            print('---------------------------------------')
            sentence = comments['message']
            
            #Translate to English
            sentence = mtranslate.translate(sentence, 'en', 'auto')
            
            #Split a single comment into multiple sentences
            sentences = sent_tokenize(sentence)
            #print(sentences)
            
            #Sentiment for each sentence
            for s in sentences:
                
                s = s.lower()
                s = s.replace('chi fitness', ' ')
                s = s.replace('chi', ' ')
                s = re.sub(r"\byin\b", "", s)
                s = re.sub(r"\byang\b", "", s)
                
                print("Sentence: ", s)
                sentiment = 0.0
                #print(s)
                
                tokenized = word_tokenize(s)
                #print(tokenized)
                
                new_tokenized = [x for x in tokenized if x not in weird_sentiment_words]
                tokenized = new_tokenized
                
                #POS tag each word
                tagged = pos_tag(tokenized)
                print(tagged)
                
                previous_word = ""
                list_of_nouns = []
                
                #Get the sentiment for each word
                for w, t in tagged:
                    
                    
                    sentiment_added = False
                    
                    #Convert Penn Treebank POS tag to Wordnet POS tag
                    wn_tag = penn_to_wn(t)
                    
                    #If successfully converted, lemmatize using the Wordnet POS tag
                    #If not successfully converted, lemmatize without any POS tags
                    if wn_tag is None:
                        lemma = lemmatizer.lemmatize(w)
                    else:
                        lemma = lemmatizer.lemmatize(w, pos=wn_tag)
                        
                    #If can't lemmatize, set the word to be passed into SentiWordnet as the initial word, w
                    #If can lemmatize, set the word to be passed into Sentiwordnet as the lemmatized word, lemma
                    if not lemma:
                        print("Can't lemmatize: ", w)
                        final_word = w
                    else:
                        print("Lemmatized word: ", lemma)
                        final_word = lemma
                        
                    #Make cool and wow positive words
                    if (final_word in ['cool', 'wow', 'glad']) and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['cool', 'wow', 'glad', 'lose', 'lost']):
                        print(final_word, " has a score of 0.5")
                        sentiment = sentiment + 0.5
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice') and (previous_word == 'not'):
                        print('Not is the previous word')
                        sentiment = sentiment - 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word == 'nice'):
                        print(final_word, " has a score of 0.875")
                        sentiment = sentiment + 0.875
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']) and (previous_word == 'not'):
                        sentiment = sentiment + 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                    elif (final_word in ['terminate', 'stop', 'halt', 'cancel']):
                        sentiment = sentiment - 0.75
                        print("The word terminate, stop or halt is detected and the sentiment is -0.75")
                        previous_word = final_word
                        sentiment_added = True
                        
                    #If the Wordnet POS tag exists, use it to get the synset of the word
                    if wn_tag is None:
                        synsets = wn.synsets(final_word)
                    else:
                        synsets = wn.synsets(final_word, pos=wn_tag)
                        
                    #If no synsets found, skip the word
                    if not synsets:
                        print("Synset not found")
                        continue
                        
                    # Take the first sense of the word which is the most common
                    synset = synsets[0]
                    swn_synset = swn.senti_synset(synset.name())
                    
                    if sentiment_added == False:
                        if final_word != 'fitness' and previous_word == 'not':      
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment - swn_synset.pos_score() - swn_synset.neg_score()
                        elif final_word != 'fitness':
                            print(final_word, " | ", synset, " | ", swn_synset, " | ", synset.definition())
                            sentiment = sentiment + swn_synset.pos_score() - swn_synset.neg_score()
                        else:
                            print("The word fitness is not considered as a sentiment")
                    
                    previous_word = final_word
                    
                    #Extract nouns
                    if t.startswith('N') or w == 'post':
                        if w != 'i' and w != 'la' and w != 'hi':
                            list_of_nouns.append(w)
                        
                try:
                    df2 = pd.DataFrame([[line['message'], s, ','.join(list_of_nouns), sentiment, 'Facebook Comment Chi Fitness']],columns=['Post', 'Comment', 'Topic', 'Sentiment', 'Source'])
                except Exception:
                    df2 = pd.DataFrame([['No message', s, ','.join(list_of_nouns), sentiment, 'Facebook Comment Chi Fitness']],columns=['Post', 'Comment', 'Topic', 'Sentiment', 'Source'])
                    
                df = pd.concat([df2,df], ignore_index=True)
                    
                # sum greater than 0 => positive sentiment
                if sentiment >= 0:
                    print("This sentence has a positive sentiment: " + str(sentiment))
                elif sentiment < 0:
                    print("This sentence has a negative sentiment: " + str(sentiment))
        print()
    except:
        continue

New comment
---------------------------------------
Sentence:  my beloved, all of this.
[('my', 'PRP$'), ('beloved', 'VBN'), (',', ','), ('all', 'DT'), ('of', 'IN'), ('this', 'DT'), ('.', '.')]
Lemmatized word:  my
Synset not found
Lemmatized word:  beloved
Synset not found
Lemmatized word:  ,
Synset not found
Lemmatized word:  all
all  |  Synset('all.a.01')  |  <all.a.01: PosScore=0.0 NegScore=0.0>  |  quantifier; used with either mass or count nouns to indicate the whole number or amount of or every one of a class
Lemmatized word:  of
Synset not found
Lemmatized word:  this
Synset not found
Lemmatized word:  .
Synset not found
This sentence has a positive sentiment: 0.0
Sentence:  i just like it.
[('just', 'RB'), ('like', 'IN'), ('it', 'PRP'), ('.', '.')]
Lemmatized word:  just
just  |  Synset('merely.r.01')  |  <merely.r.01: PosScore=0.0 NegScore=0.0>  |  and nothing more
Lemmatized word:  like
like  |  Synset('like.n.01')  |  <like.n.01: PosScore=0.125 NegScore=0.0>  |  a similar k

In [19]:
#df.to_json('chi_fitness_comments_sentiment.json')

In [25]:
df_filtered = df[df['Sentiment'] != 0]
print(len(df_filtered[df_filtered['Sentiment'] < 0]))
print(len(df_filtered[df_filtered['Sentiment'] > 0]))

38
137


In [26]:
df_filtered[df_filtered['Sentiment'] > 0]

,Post,Comment,Topic,Sentiment,Source
1,An hour of workout is only 4% of your day! #No...,really wanna join ...,join,0.125,Facebook Comment Chi Fitness
3,An hour of workout is only 4% of your day! #No...,anything about is very important,,1.000,Facebook Comment Chi Fitness
4,An hour of workout is only 4% of your day! #No...,upgrading the system is like forever.,system,0.750,Facebook Comment Chi Fitness
8,An hour of workout is only 4% of your day! #No...,many times already.,times,0.125,Facebook Comment Chi Fitness
14,An hour of workout is only 4% of your day! #No...,cool...,cool,0.500,Facebook Comment Chi Fitness
15,An hour of workout is only 4% of your day! #No...,anyone else love this post as much as me #tr...,post,0.500,Facebook Comment Chi Fitness
16,"It's Friday, but we hustle like it's Monday! ...",posts like this are why everyone likes facebook,posts,0.375,Facebook Comment Chi Fitness
17,"It's Friday, but we hustle like it's Monday! ...",posts about great fitness advice are why eve...,"posts,advice",0.250,Facebook Comment Chi Fitness
20,"It's Friday, but we hustle like it's Monday! ...",amazing post very interesting,post,0.125,Facebook Comment Chi Fitness
21,"It's Friday, but we hustle like it's Monday! ...",thanks for the info,"thanks,info",0.125,Facebook Comment Chi Fitness


In [27]:
df_filtered[df_filtered['Sentiment'] < 0]

,Post,Comment,Topic,Sentiment,Source
0,An hour of workout is only 4% of your day! #No...,never did imagine how many people have been to...,"people,loss,issues",-0.500,Facebook Comment Chi Fitness
6,An hour of workout is only 4% of your day! #No...,"when the problem occurs, you will rise up the ...","problem,reasons",-0.500,Facebook Comment Chi Fitness
7,An hour of workout is only 4% of your day! #No...,do not tell me you are still in the process of...,"process,system",-0.125,Facebook Comment Chi Fitness
9,An hour of workout is only 4% of your day! #No...,this was not first time happen to me.,time,-0.500,Facebook Comment Chi Fitness
10,An hour of workout is only 4% of your day! #No...,i cannot plan my time for workout if i cant ac...,"time,workout,access,timetable",-0.625,Facebook Comment Chi Fitness
11,An hour of workout is only 4% of your day! #No...,i am so stressful with this problem.,problem,-1.125,Facebook Comment Chi Fitness
13,An hour of workout is only 4% of your day! #No...,"hi, i am still having problem to access timeta...","problem,access,class,website",-0.375,Facebook Comment Chi Fitness
19,"It's Friday, but we hustle like it's Monday! ...",they make dropping weight seem awful simple here.,"weight,simple",-0.875,Facebook Comment Chi Fitness
30,CHi Fitness is proud to be Malaysian made! Cel...,or the next month still rm60?,month,-0.125,Facebook Comment Chi Fitness
35,CHi Fitness is proud to be Malaysian made! Cel...,jing yuan so cheap,yuan,-0.250,Facebook Comment Chi Fitness


In [23]:
df_filtered.to_json('chifitnesscommentsentiment.json', orient='records')